# Step-Forward encoder testing

In [1]:
from dataset import CHBMITDataset

data_path = "./CHB-MIT/processed"
dataset = CHBMITDataset(data_path)

In [2]:
import torch

from utils.preprocess import VectorizeSTFT

stft_data = VectorizeSTFT(dataset.data)
magnitudes = torch.abs(stft_data)
# angles = torch.angle(stft_data)

In [3]:
del dataset

In [4]:
del stft_data

In [5]:
def normalize(x: torch.Tensor) -> torch.Tensor:
    x_min = x.min(dim=-1, keepdim=True).values
    x_max = x.max(dim=-1, keepdim=True).values

    diff = x_max - x_min
    diff[diff == 0] = 1.0

    return (x - x_min) / diff

In [6]:
normalize_magnitude = normalize(magnitudes)

In [7]:
import optuna
from encoder import StepForwardEncoder

def objective(trial: optuna.Trial) -> float:

    threshold = trial.suggest_float("threshold", 0.1, 0.99)
    sf_encoder = StepForwardEncoder(threshold=threshold)

    base = normalize_magnitude.select(-1, 0).unsqueeze(-1)
    encoded_data = sf_encoder.encode(normalize_magnitude)
    decoded_data = sf_encoder.decode(encoded_data, base)

    mse = torch.nn.functional.mse_loss(decoded_data, normalize_magnitude).item()
    return mse

In [8]:
from config import DB_CONFIG

study_name = "Step Forward Encoder"
storage_url = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

In [9]:
study = optuna.create_study(
    direction="minimize",
    study_name=study_name,
    storage=storage_url,
    load_if_exists=True,
)

[I 2025-04-27 14:50:11,953] Using an existing study with name 'Step Forward Encoder' instead of creating a new one.


In [10]:
study.optimize(objective, n_trials=50)

[I 2025-04-27 14:50:19,436] Trial 5 finished with value: 0.10077301412820816 and parameters: {'threshold': 0.8279260676490113}. Best is trial 5 with value: 0.10077301412820816.
[I 2025-04-27 14:50:23,920] Trial 6 finished with value: 0.13062284886837006 and parameters: {'threshold': 0.9790798903050332}. Best is trial 5 with value: 0.10077301412820816.
[I 2025-04-27 14:50:31,579] Trial 7 finished with value: 0.0222177654504776 and parameters: {'threshold': 0.11843614847590299}. Best is trial 7 with value: 0.0222177654504776.
[I 2025-04-27 14:50:36,075] Trial 8 finished with value: 0.08270931243896484 and parameters: {'threshold': 0.6880532708036667}. Best is trial 7 with value: 0.0222177654504776.
[I 2025-04-27 14:50:40,340] Trial 9 finished with value: 0.12956705689430237 and parameters: {'threshold': 0.9756321660060117}. Best is trial 7 with value: 0.0222177654504776.
[I 2025-04-27 14:50:44,987] Trial 10 finished with value: 0.04188653454184532 and parameters: {'threshold': 0.47147256

In [11]:
study.best_params

{'threshold': 0.1965616735274163}